In [1]:
import pandas as pd
import numpy as np

In [21]:
file_path = 'data/Percentage Project Example.xlsx'
dfs = pd.read_excel(file_path, sheet_name=None)

In [6]:
dfs

{'Summary':                   Q1.What year are you in?           Unnamed: 1
 0                           Answer Choices  Response Percentage
 1                               First year               36.84%
 2                                Sophomore               33.83%
 3                                   Junior               14.29%
 4                                   Senior               12.03%
 ..                                     ...                  ...
 220  Q29.Are you an international student?                  NaN
 221                         Answer Choices  Response Percentage
 222                                    Yes               21.80%
 223                                     No               77.44%
 224                      Prefer not to say                0.75%
 
 [225 rows x 2 columns],
 'Gender':                            Gender Demographic Analysis  \
 0    * Note: Some demographic groups had less than ...   
 1     Q3.Check all of the following that apply to you

In [9]:
for sheet_name,_ in dfs.items():
    print(sheet_name)

Summary
Gender
Race&Ethnicity
Disability
LGBQ+
Transgender
First Gen
Low Income
International


In [32]:
df = pd.read_excel(file_path, sheet_name='Gender')

# questions = df.iloc[0]
# response_categories = df.iloc[:, 0]
df.head()

,Gender Demographic Analysis,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,* Note: Some demographic groups had less than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q3.Check all of the following that apply to you:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"I feel pressure at Brown to find internships, ...",I feel confident studying computer science and...,I feel intimidated studying Computer Science a...,None of the above,NaN,NaN,NaN
3,Woman,86.44%,30.51%,69.49%,1.69%,NaN,NaN,NaN
4,Man,68.57%,75.71%,41.43%,1.43%,NaN,NaN,NaN


In [36]:
def extract_data_groups(file_path):
    # Read the excel file
    df = pd.read_excel(file_path, sheet_name='Gender')
    
    # Dictionary to hold the extracted groups
    groups = {}
    current_key = None
    start_index = None
    
    # Iterate through the DataFrame to find rows with 'Q'
    for index, row in df.iterrows():
        # Check if any cell in the row contains 'Q'
        if row.astype(str).str.contains('Q').any():
            # If we have a previous key, extract the DataFrame slice up to the current row
            if current_key is not None:
                groups[current_key] = df.iloc[start_index:index]
            
            # Update the current key and start index for the next group
            current_key = row[row.astype(str).str.contains('Q')].values[0]  # Assuming only one 'Q' per key-row
            start_index = index + 1
    
    # Handle the last group if the last row with 'Q' isn't the last row of the DataFrame
    if current_key is not None and start_index < len(df):
        groups[current_key] = df.iloc[start_index:]
    
    return groups

# Uncomment to test the function
file_path = 'data/Percentage Project Example.xlsx'
data_groups = extract_data_groups(file_path)
# print(data_groups)

In [35]:
for key, df in data_groups.items():
    print(f"Key: {key}, DataFrame Shape: {df.shape}")

Key: Q3.Check all of the following that apply to you:, DataFrame Shape: (7, 8)
Key: Q4.Check all of the following that apply to you:, DataFrame Shape: (7, 8)
Key: Q5.Check all of the following that apply to you:, DataFrame Shape: (7, 8)
Key: Q6.Think about your time in CS classes. Check all of the following that apply to you:, DataFrame Shape: (7, 8)
Key: Q7.I have a faculty whom I perceive as a role model, DataFrame Shape: (7, 8)
Key: Q8.I have considered leaving my computer science-related field of study., DataFrame Shape: (7, 8)
Key: Q9.A faculty member or an administrator has encouraged me to drop out., DataFrame Shape: (7, 8)
Key: Q10.At my university, students from every background have an equal chance to succeed., DataFrame Shape: (7, 8)
Key: Q11.I feel adequately supported by the CS department and the resources offered by the department., DataFrame Shape: (7, 8)
Key: Q12.I  feel comfortable using Brown Computer Science physical spaces (sunlab, CIT lobby, atrium, lecture halls, 

0
Gender Demographic Analysis    * Note: Some demographic groups had less than ...
Unnamed: 1                                                                   NaN
Unnamed: 2                                                                   NaN
Unnamed: 3                                                                   NaN
Unnamed: 4                                                                   NaN
Unnamed: 5                                                                   NaN
Unnamed: 6                                                                   NaN
Unnamed: 7                                                                   NaN
Name: 0, dtype: object
1
Gender Demographic Analysis    Q3.Check all of the following that apply to you:
Unnamed: 1                                                                  NaN
Unnamed: 2                                                                  NaN
Unnamed: 3                                                                  NaN
Unnam

In [27]:
all_data = []
# df = pd.read_excel(file_path, sheet_name=sheet_name)
df.columns = df.iloc[0]  # Set the first row as header
df = df.drop(0).reset_index(drop=True)  # Drop the header row from the data

# Melt the DataFrame to long format
melted_df = df.melt(id_vars=df.columns[0], var_name='Demographic', value_name='Value')
melted_df.rename(columns={df.columns[0]: 'Question'}, inplace=True)
all_data.append(melted_df)
all_data

[                                              Question Demographic  \
 0     Q3.Check all of the following that apply to you:         NaN   
 1                                                  NaN         NaN   
 2                                                Woman         NaN   
 3                                                  Man         NaN   
 4                                           Non-binary         NaN   
 ...                                                ...         ...   
 1052                                             Woman         NaN   
 1053                                               Man         NaN   
 1054                                        Non-binary         NaN   
 1055                          Prefer to self-describe:         NaN   
 1056                                 Prefer not to say         NaN   
 
                                                   Value  
 0                                                   NaN  
 1     I feel pressure at Br

In [13]:
df

,Gender Demographic Analysis,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,* Note: Some demographic groups had less than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q3.Check all of the following that apply to you:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"I feel pressure at Brown to find internships, ...",I feel confident studying computer science and...,I feel intimidated studying Computer Science a...,None of the above,NaN,NaN,NaN
3,Woman,86.44%,30.51%,69.49%,1.69%,NaN,NaN,NaN
4,Man,68.57%,75.71%,41.43%,1.43%,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
147,Woman,66.67%,29.41%,3.92%,NaN,NaN,NaN,NaN
148,Man,43.10%,37.93%,18.97%,NaN,NaN,NaN,NaN
149,Non-binary,*,*,*,NaN,NaN,NaN,NaN
150,Prefer to self-describe:,*,*,*,NaN,NaN,NaN,NaN


In [12]:
questions

[nan, nan, nan, nan, nan, nan, nan]

In [10]:
df = pd.read_excel(file_path, sheet_name='Gender')

# Parse out the questions from the first row and the response options from the first column, assuming they are there
questions = df.iloc[0, 1:].tolist()  # Assuming questions are in the first row, skipping the first cell
response_options = df.iloc[1:, 0].tolist()  # Assuming response options are in the first column, skipping the header

# Create a new DataFrame with response options as rows and demographics as columns
processed_data = {
    (question, response_option): df[response_option].values
    for question, response_option in zip(questions, response_options)
}

# Convert the dictionary to a DataFrame
processed_df = pd.DataFrame(processed_data)
processed_df

KeyError: 'Q3.Check all of the following that apply to you:'

In [5]:
# Initialize a container for your combined DataFrame and README content
combined_df = None
readme_content = []

# Process each sheet
for sheet_name, df in dfs.items():
    # Assume the first row contains the question text
    questions = df.iloc[0]
    # Assume the first column contains the response categories
    response_categories = df.iloc[:, 0]
    
    # Transform the DataFrame from wide to long format
    melted_df = df.melt(id_vars=response_categories, var_name='Demographic', value_name='Percentage')
    
    # Add the melted_df to the combined DataFrame
    combined_df = pd.concat([combined_df, melted_df], axis=0)
    
    # Add questions to the README content
    for question_number, question_text in enumerate(questions, start=1):
        readme_content.append(f"Q{question_number}: {question_text}")

# Save the combined DataFrame to a CSV file
combined_df.to_csv('combined_survey_data.csv', index=False)

KeyError: 'The following \'id_vars\' are not present in the DataFrame: [\'African American/Black\', \'Agree\', \'Answer Choices\', \'Applied Math\', \'Applied Math-CS\', \'Asian\', \'CS-Economics\', \'Computational Biology\', \'Computer Science\', \'Disagree\', \'Engineering\', \'First year\', \'Graduate Student\', \'Heterosexual/straight\', \'Hispanic/Latinx\', \'I ask questions on EdStem, anonymously or publicly.\', \'I feel confident studying computer science and related fields.\', \'I feel intimidated studying Computer Science and related fields.\', \'I feel more comfortable in a class taught by a professor who shares a similar identity, such as race, ethnicity, gender, gender expression, age, disability, sexual orientation, parental education, or income status.\', \'I feel pressure at Brown to find internships, job opportunities, and/or extracurricular activities.\', \'I generally feel comfortable asking questions during lecture.\', \'I generally feel comfortable asking questions 